In [114]:
import pandas as pd
import numpy as np

In [115]:
credits_data = pd.read_csv('csv-files\\credits.csv')
meta = pd.read_csv('csv-files\\movie_metadata_2.csv')

C:\Users\91635\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [116]:
credits_data.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [117]:
meta.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [118]:
meta.shape


(45466, 24)

### Changing format of release_date column

In [119]:
meta['release_date']=pd.to_datetime(meta['release_date'],errors = 'coerce')

In [120]:
meta['year'] = meta['release_date'].dt.year

In [121]:
meta['year'].value_counts().sort_index()

1874.0       1
1878.0       1
1883.0       1
1887.0       1
1888.0       2
          ... 
2015.0    1905
2016.0    1604
2017.0     532
2018.0       5
2020.0       1
Name: year, Length: 135, dtype: int64

### As we already have data till 2016 , we only need data after 2016

In [122]:
new_data = meta[meta['year'] == 2017]

In [123]:
new_data = new_data[['id','genres','overview','title','year']]

In [124]:
new_data.head(1)

,id,genres,overview,title,year
26560,166426,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","Thrust into an all-new paycheck, a down-on-his...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0


In [125]:
new_data.shape

(532, 5)

### Joining the credits_data and new_data dataframe

In [126]:
new_data['id'] = new_data['id'].astype(int)


In [127]:
data = new_data.merge(credits_data,on = 'id')

In [128]:
data.head(1)

,id,genres,overview,title,year,cast,crew
0,166426,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","Thrust into an all-new paycheck, a down-on-his...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."


### Handling the cast column


In [129]:
import ast

- converting the dictionary format into list format and selecting only top three actors/actresses

In [130]:
def convert(obj):
  s = ""
  counter =0
  for i in ast.literal_eval(obj):
    if counter != 3:
      s += i['name'] + ","
      counter+=1
    else:
      break
  if s == "":
    return np.NaN
  return s

In [131]:
data['cast'] = data['cast'].apply(convert)

In [132]:
data.head(3)

,id,genres,overview,title,year,cast,crew
0,166426,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","Thrust into an all-new paycheck, a down-on-his...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0,"Johnny Depp,Javier Bardem,Geoffrey Rush,","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."
1,141052,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Fueled by his restored faith in humanity and i...,Justice League,2017.0,"Ben Affleck,Henry Cavill,Gal Gadot,","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de..."
2,284053,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Thor is imprisoned on the other side of the un...,Thor: Ragnarok,2017.0,"Chris Hemsworth,Tom Hiddleston,Cate Blanchett,","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de..."


In [133]:
data.shape

(531, 7)

In [134]:
data['cast'] = data['cast'].astype(str)

In [135]:
data['cast']=data['cast'].apply(lambda x:x.replace(" ",""))
    

### Handling genres column

In [136]:
data['genres'][0]

"[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 14, 'name': 'Fantasy'}, {'id': 35, 'name': 'Comedy'}]"

- converting the dictionary format into list format

In [137]:
def convert2(obj):
  l =[]
  for i in ast.literal_eval(obj):
    l.append(i['name'])
  if l == []:
    return np.NaN
  return l
    

In [138]:
data['genres'] = data['genres'].apply(convert2)

In [139]:
data.head(1)

,id,genres,overview,title,year,cast,crew
0,166426,"[Adventure, Action, Fantasy, Comedy]","Thrust into an all-new paycheck, a down-on-his...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0,"JohnnyDepp,JavierBardem,GeoffreyRush,","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."


### Handling overview column

In [140]:
data['overview'][0]

"Thrust into an all-new paycheck, a down-on-his-luck Capt. Jack Sparrow feels the winds of ill-fortune blowing even more strongly when deadly ghost sailors led by his old nemesis, the evil Capt. Salazar, escape from the Devil's Triangle. Jack's only hope of a payout lies in seeking out the legendary Trident of Numbers, but to find it, he must forge an uneasy alliance with a reasonably intelligent and pretty astronomer and a irritating young man in the British navy."

In [141]:
data['overview'] = data['overview'].astype(str)

In [142]:
data['overview'] = data['overview'].apply(lambda x:x.split())

In [143]:
data.head(1)

,id,genres,overview,title,year,cast,crew
0,166426,"[Adventure, Action, Fantasy, Comedy]","[Thrust, into, an, all-new, paycheck,, a, down...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0,"JohnnyDepp,JavierBardem,GeoffreyRush,","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."


### Handling crew columns

- only fetching director's name

In [144]:
def convert3(obj):
  s = ""
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director':
      s += i['name']
      break
  return s

In [145]:
data['crew'] = data['crew'].apply(convert3)

In [146]:
data.head(1)

,id,genres,overview,title,year,cast,crew
0,166426,"[Adventure, Action, Fantasy, Comedy]","[Thrust, into, an, all-new, paycheck,, a, down...",Pirates of the Caribbean: Dead Men Tell No Tales,2017.0,"JohnnyDepp,JavierBardem,GeoffreyRush,",Joachim Rønning


### Removing spaces between names

In [147]:
data['crew'] = data['crew'].astype(str)

In [148]:

data['crew'] = data['crew'].apply(lambda x:x.replace(" ",""))


### Drop Null values

In [150]:
data.isna().sum()

id          0
genres      7
overview    0
title       0
year        0
cast        0
crew        0
dtype: int64

In [151]:
data = data.dropna(how='any')

In [152]:
data.isna().sum()


id          0
genres      0
overview    0
title       0
year        0
cast        0
crew        0
dtype: int64

### Lowering all movie titles

In [153]:
data['title'] = data['title'].apply(lambda x:x.lower())

In [154]:
data.head(1)

,id,genres,overview,title,year,cast,crew
0,166426,"[Adventure, Action, Fantasy, Comedy]","[Thrust, into, an, all-new, paycheck,, a, down...",pirates of the caribbean: dead men tell no tales,2017.0,"JohnnyDepp,JavierBardem,GeoffreyRush,",JoachimRønning


### Combining all tags

In [155]:
data['cast'] = data['cast'].apply(lambda x:x.split())
data['crew'] = data['crew'].apply(lambda x:x.split())


In [156]:
data['crew'][0]

['JoachimRønning']

In [157]:
data['tags'] = data['cast'] + data['crew'] + data['genres'] + data['overview']

### Keeping only columns in need

In [158]:
data = data[['tags','title']]

In [159]:
data.head()

,tags,title
0,"[JohnnyDepp,JavierBardem,GeoffreyRush,, Joachi...",pirates of the caribbean: dead men tell no tales
1,"[BenAffleck,HenryCavill,GalGadot,, ZackSnyder,...",justice league
2,"[ChrisHemsworth,TomHiddleston,CateBlanchett,, ...",thor: ragnarok
3,"[ChrisPratt,ZoeSaldana,DaveBautista,, JamesGun...",guardians of the galaxy vol. 2
4,"[PierceBrosnan,WilliamHurt,BenjaminWalker,, Se...",the king's daughter


In [160]:
data['tags']=data['tags'].apply(lambda x:" ".join(x))

In [161]:
data['tags'] = data['tags'].apply(lambda x:x.lower())

In [166]:
data['tags'] = data['tags'].apply(lambda x:x.replace(","," "))

### Reading movie data till 2016

In [168]:
old = pd.read_csv('csv-files\\till_2016.csv')
old.head()

,tags,title
0,cchpounder joeldavidmoore wesstudi jamescamero...,avatar
1,johnnydepp orlandobloom jackdavenport goreverb...,pirates of the caribbean: at world's end
2,christophwaltz rorykinnear stephaniesigman sam...,spectre
3,tomhardy christianbale josephgordon-levitt chr...,the dark knight rises
4,dougwalker robwalker unknown dougwalker docume...,star wars: episode vii - the force awakens ...


### Appending old data with 2017 data

In [169]:
new = old.append(data)

In [170]:
new

,tags,title
0,cchpounder joeldavidmoore wesstudi jamescamero...,avatar
1,johnnydepp orlandobloom jackdavenport goreverb...,pirates of the caribbean: at world's end
2,christophwaltz rorykinnear stephaniesigman sam...,spectre
3,tomhardy christianbale josephgordon-levitt chr...,the dark knight rises
4,dougwalker robwalker unknown dougwalker docume...,star wars: episode vii - the force awakens ...
...,...,...
525,adelaidekane benjaminhollingsworth jeanlouisak...,can't buy my love
526,inkahaapamäki rosahonkonen tiitusrantala hann...,thick lashes of lauri mäntyvaara
527,loudiamondphillips wallaceshawn ginaholden jo...,cop and a half: new recruit
528,nan bethdavid family animation romance comedy ...,in a heartbeat


In [171]:
new.shape,data.shape

((5567, 2), (524, 2))

### Drop Duplicates

In [172]:
new['title'].duplicated().sum()

138

In [173]:
new.drop_duplicates(subset = 'title',keep='last',inplace=True)

In [174]:
new.shape

(5429, 2)

In [175]:
new.to_csv('csv-files\\df_2017.csv',index=False)